In [2]:
### Transfer Learning- Model is trained here on imagenet dataset and we transfer the learning to predict on the different dataset.

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds

In [4]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

In [5]:
train_ds[0].shape

TensorShape([442, 1024, 3])

In [6]:
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [7]:
train_ds[0]

<tf.Tensor: shape=(150, 150, 3), dtype=float32, numpy=
array([[[105.833336, 132.12001 ,  35.522312],
        [115.6264  , 138.69307 ,  42.374935],
        [ 99.71022 , 125.11689 ,  30.989334],
        ...,
        [  0.      ,   0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      ]],

       [[123.5656  , 141.052   ,  37.0528  ],
        [133.22081 , 149.3816  ,  58.62    ],
        [100.989334, 129.396   ,  40.490665],
        ...,
        [  0.      ,   0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      ]],

       [[126.98178 , 139.612   ,  27.463554],
        [137.352   , 152.64134 ,  63.65867 ],
        [115.04001 , 142.34001 ,  51.902225],
        ...,
        [  0.      ,   0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      ],
        [  0.      ,   0.      ,   0.      ]],

       ...,

       [[  0.      ,   0.      ,   0.      ],
        [  0.

In [8]:
train_labels

<tf.Tensor: shape=(2569,), dtype=int64, numpy=array([2, 3, 3, ..., 0, 2, 0], dtype=int64)>

In [9]:
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [10]:
train_labels

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

# Pretrained VGG16 Image Classification model

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [12]:
train_ds[0].shape

TensorShape([150, 150, 3])

In [13]:
train_ds.shape

TensorShape([2569, 150, 150, 3])

In [14]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
#base_model = VGG16(weights="imagenet", include_top=False, input_shape=(150,150,3))

#both are same

In [15]:
## will not train base mode
base_model.trainable = False 

In [16]:
## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [17]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

 block3_pool (MaxPooling2D)  (None, 18, 18, 256)       0         
                                                                 
 block4_conv1 (Conv2D)       (None, 18, 18, 512)       1180160   
                                                                 
 block4_conv2 (Conv2D)       (None, 18, 18, 512)       2359808   
                                                                 
 block4_conv3 (Conv2D)       (None, 18, 18, 512)       2359808   
                                                                 
 block4_pool (MaxPooling2D)  (None, 9, 9, 512)         0         
                                                                 
 block5_conv1 (Conv2D)       (None, 9, 9, 512)         2359808   
                                                                 
 block5_conv2 (Conv2D)       (None, 9, 9, 512)         2359808   
                                                                 
 block5_conv3 (Conv2D)       (None, 9, 9, 512)         2359808   
          

In [18]:
#add our layers on top of this model
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,                     #added VGG16 base model
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [20]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)  #avoid overfitting and if the model is not improving, it will stop at that level

In [21]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 246s 4s/step - loss: 1.8143 - accuracy: 0.5027 - val_loss: 1.1066 - val_accuracy: 0.6304
Epoch 2/50
65/65 [==============================] - 228s 4s/step - loss: 0.7416 - accuracy: 0.7518 - val_loss: 0.9333 - val_accuracy: 0.7004
Epoch 3/50
65/65 [==============================] - 225s 3s/step - loss: 0.5066 - accuracy: 0.8161 - val_loss: 0.9262 - val_accuracy: 0.7121
Epoch 4/50
65/65 [==============================] - 223s 3s/step - loss: 0.3068 - accuracy: 0.9002 - val_loss: 1.0042 - val_accuracy: 0.6984
Epoch 5/50
65/65 [==============================] - 221s 3s/step - loss: 0.2080 - accuracy: 0.9358 - val_loss: 1.0497 - val_accuracy: 0.6946
Epoch 6/50
65/65 [==============================] - 221s 3s/step - loss: 0.1536 - accuracy: 0.9518 - val_loss: 1.0430 - val_accuracy: 0.6965
Epoch 7/50
65/65 [==============================] - 221s 3s/step - loss: 0.1200 - accuracy: 0.9606 - val_loss: 1.1266 - val_accuracy: 0.6965
Epoch 8/50
65

# VGG16 From Scratch

In [22]:

from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


full_model = Sequential()
full_model.add(Rescaling(1./255, input_shape=(150,150,3)))

full_model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
full_model.add(layers.MaxPooling2D(3))

full_model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
full_model.add(layers.MaxPooling2D(2))

full_model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
full_model.add(layers.MaxPooling2D(2))

full_model.add(layers.Flatten())
full_model.add(layers.Dense(50, activation='relu'))
full_model.add(layers.Dense(20, activation='relu'))
full_model.add(layers.Dense(5, activation='softmax'))


full_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

full_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 20s 273ms/step - loss: 1.5272 - accuracy: 0.2876 - val_loss: 1.4670 - val_accuracy: 0.3541
Epoch 2/50
65/65 [==============================] - 15s 233ms/step - loss: 1.3720 - accuracy: 0.3781 - val_loss: 1.2481 - val_accuracy: 0.4572
Epoch 3/50
65/65 [==============================] - 16s 245ms/step - loss: 1.2278 - accuracy: 0.4710 - val_loss: 1.2123 - val_accuracy: 0.5000
Epoch 4/50
65/65 [==============================] - 15s 233ms/step - loss: 1.1650 - accuracy: 0.5217 - val_loss: 1.2014 - val_accuracy: 0.4689
Epoch 5/50
65/65 [==============================] - 16s 241ms/step - loss: 1.0869 - accuracy: 0.5421 - val_loss: 1.2124 - val_accuracy: 0.4883
Epoch 6/50
65/65 [==============================] - 16s 250ms/step - loss: 1.0203 - accuracy: 0.5830 - val_loss: 1.1304 - val_accuracy: 0.5389
Epoch 7/50
65/65 [==============================] - 14s 217ms/step - loss: 0.9392 - accuracy: 0.6273 - val_loss: 1.2177 - val_accuracy: 0.5681